# Used Car Sales Analytics – End-to-End ETL Pipeline

This notebook demonstrates a complete ETL pipeline that transforms raw
used car sales data into analytics-ready fact and dimension tables.


In [36]:
# Import Required Libraries
import pandas as pd
import numpy as np
import logging
import warnings
from datetime import datetime


# SECTION 1 — Imports & Configuration

In [37]:
# File paths
RAW_DATA_PATH = "../raw/used_car_sales_raw.csv"
PROCESSED_DATA_PATH = "../processed/"
# Business constants
CURRENT_YEAR = datetime.now().year


In [38]:
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)


In [39]:
logging.info("Test log: environment setup completed")


2026-02-10 23:29:42,402 - INFO - Test log: environment setup completed


In [40]:
# Warning & Display Settings
# To keep notebook clean and readable.
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)


In [41]:
logging.info("Environment setup completed successfully")
logging.info(f"Current year set to: {CURRENT_YEAR}")


2026-02-10 23:29:42,422 - INFO - Environment setup completed successfully
2026-02-10 23:29:42,422 - INFO - Current year set to: 2026


You have successfully completed:
    ✔ Project structure
    ✔ Notebook created
    ✔ Imports
    ✔ Configuration
    ✔ Logging setup

# SECTION 2 — Extract (Read Raw Data)

In [42]:

import os

print("Current working directory:")
print(os.getcwd())

print("\nContents of current directory:")
print(os.listdir())

print("\nDoes data folder exist?")
print(os.path.exists("data"))

print("\nDoes raw CSV exist?")
print(os.path.exists("data/raw/used_car_sales_raw.csv"))


Current working directory:
c:\Users\apoorv\Desktop\used-car-etl-analytics\data\notebooks

Contents of current directory:
['used_car_analytics.db', 'used_car_etl_pipeline.ipynb']

Does data folder exist?
False

Does raw CSV exist?
False


In [43]:
#Load the Raw CSV(EXTRACT)
logging.info("Starting data extraction")
df_raw = pd.read_csv(RAW_DATA_PATH)
logging.info("Raw data loaded successfully")


2026-02-10 23:29:42,443 - INFO - Starting data extraction
2026-02-10 23:29:42,489 - INFO - Raw data loaded successfully


In [44]:
#Preview the Raw Data
df_raw.head()

,transaction_id,car_id,brand,model,manufacturing_year,car_age,fuel_type,transmission,kms_driven,listed_price,sold_price,price_difference,seller_city,seller_state,listing_date,sold_date,days_on_platform,sold_month,sold_year,is_premium_brand,car_condition,seller_rating,customer_segment,expected_market_price,price_bucket,conversion_rate,is_festive_season,sold_month_name,sold_month_num,sale_flag
0,1,2418,Bmw,X3,2015,10,Cng,Automatic,162139,949951,889590,60361,Pune,Rajasthan,14-10-2021,11-03-2021,20,11,2021,1,Poor,3.3,Mid,474975,Medium,0.11,1,Mar,3,1
1,2,7127,Hyundai,Verna,2019,6,Electric,Manual,57289,1968483,1807733,160750,Mumbai,Punjab,06-06-2021,15-07-2021,104,9,2021,0,Fair,3.3,Premium,1377938,High,0.03,0,Jul,7,1
2,3,6183,Honda,Amaze,2013,12,Electric,Manual,192873,2417596,2240026,177570,Bangalore,Maharashtra,26-12-2019,13-08-2019,63,2,2020,0,Poor,3.9,Premium,967038,High,0.03,0,Aug,8,1
3,4,4245,Maruti,Baleno,2015,10,Electric,Manual,100035,2441058,2419873,21185,Bangalore,Maharashtra,07-03-2024,13-03-2021,69,5,2024,0,Poor,3.4,Premium,1220529,High,0.06,0,Mar,3,1
4,5,2798,Toyota,Innova,2023,2,Petrol,Manual,120792,945015,813106,131909,Chandigarh,Rajasthan,21-12-2020,22-03-2020,7,12,2020,0,Excellent,4.7,Mid,850513,Medium,0.10,0,Mar,3,1


In [45]:
df_raw.tail()

,transaction_id,car_id,brand,model,manufacturing_year,car_age,fuel_type,transmission,kms_driven,listed_price,sold_price,price_difference,seller_city,seller_state,listing_date,sold_date,days_on_platform,sold_month,sold_year,is_premium_brand,car_condition,seller_rating,customer_segment,expected_market_price,price_bucket,conversion_rate,is_festive_season,sold_month_name,sold_month_num,sale_flag
7995,7996,2688,Bmw,X1,2016,9,Petrol,Automatic,155672,506587,384442,122145,Mumbai,Rajasthan,13-05-2019,13-10-2024,46,6,2019,1,Poor,3.8,Budget,278622,Low,0.04,0,Oct,10,1
7996,7997,5036,Mahindra,Scorpio,2014,11,Electric,Manual,68515,1569547,1516566,52981,Pune,Delhi,04-01-2022,04-03-2022,89,4,2022,0,Poor,3.7,Premium,706296,High,0.08,0,Mar,3,1
7997,7998,7199,Mahindra,Xuv500,2014,11,Petrol,Automatic,89093,1768833,1594112,174721,Mumbai,Delhi,09-01-2025,09-04-2025,35,2,2025,0,Poor,4.9,Premium,795974,High,0.06,0,Apr,4,1
7998,7999,1958,Mahindra,Xuv500,2022,3,Electric,Automatic,128038,805366,632208,173158,Pune,Karnataka,03-05-2024,17-08-2021,86,7,2024,0,Good,3.8,Mid,684561,Medium,0.03,0,Aug,8,1
7999,8000,6474,Maruti,Wagonr,2014,11,Diesel,Automatic,175811,1826342,1751617,74725,Bangalore,Rajasthan,31-12-2019,23-11-2025,26,1,2020,0,Poor,3.3,Premium,821853,High,0.05,0,Nov,11,1


In [46]:
#Dataset Size Overview
rows, columns = df_raw.shape
logging.info(f"Dataset contains {rows} rows and {columns} columns")


2026-02-10 23:29:42,588 - INFO - Dataset contains 8000 rows and 30 columns


# SECTION 3 — INITIAL DATA QUALITY CHECKS

In [47]:
#Inspect Column Names (Schema Understanding)
logging.info("Inspecting column names")
df_raw.columns.tolist()


2026-02-10 23:29:42,598 - INFO - Inspecting column names


['transaction_id',
 'car_id',
 'brand',
 'model',
 'manufacturing_year',
 'car_age',
 'fuel_type',
 'transmission',
 'kms_driven',
 'listed_price',
 'sold_price',
 'price_difference',
 'seller_city',
 'seller_state',
 'listing_date',
 'sold_date',
 'days_on_platform',
 'sold_month',
 'sold_year',
 'is_premium_brand',
 'car_condition',
 'seller_rating',
 'customer_segment',
 'expected_market_price',
 'price_bucket',
 'conversion_rate',
 'is_festive_season',
 'sold_month_name',
 'sold_month_num',
 'sale_flag']

In [48]:
#Check Data Types
logging.info("Checking data types")
df_raw.info()


2026-02-10 23:29:42,609 - INFO - Checking data types


<class 'pandas.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   transaction_id         8000 non-null   int64  
 1   car_id                 8000 non-null   int64  
 2   brand                  8000 non-null   str    
 3   model                  8000 non-null   str    
 4   manufacturing_year     8000 non-null   int64  
 5   car_age                8000 non-null   int64  
 6   fuel_type              8000 non-null   str    
 7   transmission           8000 non-null   str    
 8   kms_driven             8000 non-null   int64  
 9   listed_price           8000 non-null   int64  
 10  sold_price             8000 non-null   int64  
 11  price_difference       8000 non-null   int64  
 12  seller_city            8000 non-null   str    
 13  seller_state           8000 non-null   str    
 14  listing_date           8000 non-null   str    
 15  sold_date      

In [49]:
# Missing Values 
logging.info("Checking missing values")
df_raw.isnull().sum()



2026-02-10 23:29:42,634 - INFO - Checking missing values


transaction_id           0
car_id                   0
brand                    0
model                    0
manufacturing_year       0
car_age                  0
fuel_type                0
transmission             0
kms_driven               0
listed_price             0
sold_price               0
price_difference         0
seller_city              0
seller_state             0
listing_date             0
sold_date                0
days_on_platform         0
sold_month               0
sold_year                0
is_premium_brand         0
car_condition            0
seller_rating            0
customer_segment         0
expected_market_price    0
price_bucket             0
conversion_rate          0
is_festive_season        0
sold_month_name          0
sold_month_num           0
sale_flag                0
dtype: int64

In [50]:
#Duplicate rows
logging.info("Checking duplicate rows")
df_raw.duplicated().sum()

2026-02-10 23:29:42,654 - INFO - Checking duplicate rows


np.int64(0)

In [51]:
#To catch obvious business errors early.
logging.info("Running basic sanity checks")
df_raw.describe()

2026-02-10 23:29:42,689 - INFO - Running basic sanity checks


,transaction_id,car_id,manufacturing_year,car_age,kms_driven,listed_price,sold_price,price_difference,days_on_platform,sold_month,sold_year,is_premium_brand,seller_rating,expected_market_price,conversion_rate,is_festive_season,sold_month_num,sale_flag
count,8000.00000,8000.000000,8000.000000,8000.000000,8000.000000,8.000000e+03,8.000000e+03,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000,8.000000e+03,8000.000000,8000.000000,8000.00000,8000.0
mean,4000.50000,5539.103000,2018.063250,6.936750,102774.136250,1.341170e+06,1.251477e+06,89693.272875,60.745000,6.500625,2021.698500,0.254375,4.056138,8.764638e+05,0.062086,0.165750,6.55625,1.0
std,2309.54541,2577.116592,3.739786,3.739786,56096.164226,6.621348e+05,6.651735e+05,51967.571071,34.044197,3.457889,1.760322,0.435537,0.494214,5.153419e+05,0.030294,0.371879,3.44954,0.0
min,1.00000,1001.000000,2012.000000,1.000000,5056.000000,2.000180e+05,2.132400e+04,73.000000,3.000000,1.000000,2019.000000,0.000000,3.200000,7.275100e+04,0.020000,0.000000,1.00000,1.0
25%,2000.75000,3340.750000,2015.000000,4.000000,54264.000000,7.693350e+05,6.764720e+05,44859.500000,31.000000,4.000000,2020.000000,0.000000,3.600000,4.529952e+05,0.040000,0.000000,4.00000,1.0
50%,4000.50000,5561.000000,2018.000000,7.000000,103192.500000,1.329258e+06,1.240210e+06,89531.500000,61.000000,7.000000,2022.000000,0.000000,4.100000,7.940325e+05,0.060000,0.000000,7.00000,1.0
75%,6000.25000,7744.000000,2021.000000,10.000000,150942.000000,1.912066e+06,1.820921e+06,134009.250000,90.000000,10.000000,2023.000000,1.000000,4.500000,1.222026e+06,0.070000,0.000000,10.00000,1.0
max,8000.00000,9999.000000,2024.000000,13.000000,199994.000000,2.498760e+06,2.493148e+06,179985.000000,120.000000,12.000000,2025.000000,1.000000,4.900000,2.373233e+06,0.150000,1.000000,12.00000,1.0


You have now:✔ Loaded raw data
✔ Understood schema
✔ Checked data types
✔ Identified missing values
✔ Looked for duplicates
✔ Performed sanity checks

# SECTION 4 — CLEAN & STANDARDIZE

In [52]:
#Create a Working Copy + Standardize Column Names
logging.info("Creating working copy of raw data")
df = df_raw.copy()
logging.info("Standardizing column names")
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

2026-02-10 23:29:42,771 - INFO - Creating working copy of raw data
2026-02-10 23:29:42,775 - INFO - Standardizing column names


In [53]:
#Handle Missing Values (Basic Strategy)
logging.info("Handling missing values")
missing_before = df.isnull().sum().sum()
df = df.dropna()
missing_after = df.isnull().sum().sum()
logging.info(f"Missing values before cleaning: {missing_before}")
logging.info(f"Missing values after cleaning: {missing_after}")

2026-02-10 23:29:42,784 - INFO - Handling missing values
2026-02-10 23:29:42,797 - INFO - Missing values before cleaning: 0
2026-02-10 23:29:42,797 - INFO - Missing values after cleaning: 0


In [54]:
#Remove Duplicate Rows
logging.info("Removing duplicate rows")
rows_before = df.shape[0]
df = df.drop_duplicates()
rows_after = df.shape[0]
logging.info(f"Rows before deduplication: {rows_before}")
logging.info(f"Rows after deduplication: {rows_after}")

2026-02-10 23:29:42,805 - INFO - Removing duplicate rows
2026-02-10 23:29:42,822 - INFO - Rows before deduplication: 8000
2026-02-10 23:29:42,823 - INFO - Rows after deduplication: 8000


In [55]:
#Post-Cleaning Validation
logging.info("Post-cleaning data quality check")
df.isnull().sum(), df.duplicated().sum()


2026-02-10 23:29:42,832 - INFO - Post-cleaning data quality check


(transaction_id           0
 car_id                   0
 brand                    0
 model                    0
 manufacturing_year       0
 car_age                  0
 fuel_type                0
 transmission             0
 kms_driven               0
 listed_price             0
 sold_price               0
 price_difference         0
 seller_city              0
 seller_state             0
 listing_date             0
 sold_date                0
 days_on_platform         0
 sold_month               0
 sold_year                0
 is_premium_brand         0
 car_condition            0
 seller_rating            0
 customer_segment         0
 expected_market_price    0
 price_bucket             0
 conversion_rate          0
 is_festive_season        0
 sold_month_name          0
 sold_month_num           0
 sale_flag                0
 dtype: int64,
 np.int64(0))

# Section 5: Transform & Feature Engineering

In [56]:
# Fix Numeric Columns (Type Safety)
logging.info("Converting numeric columns to proper types")
numeric_columns = [
    "listed_price",
    "sold_price",
    "expected_market_price",
    "kms_driven",
    "conversion_rate"
]
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors="coerce")


2026-02-10 23:29:42,865 - INFO - Converting numeric columns to proper types


In [57]:
# Fix Date Columns
logging.info("Converting date columns")
df["listing_date"] = pd.to_datetime(df["listing_date"], errors="coerce")
df["sold_date"] = pd.to_datetime(df["sold_date"], errors="coerce")


2026-02-10 23:29:42,876 - INFO - Converting date columns


In [58]:
# Create Business Feature: Vehicle Age
logging.info("Creating vehicle_age feature")
df["vehicle_age"] = CURRENT_YEAR - df["manufacturing_year"]

2026-02-10 23:29:42,917 - INFO - Creating vehicle_age feature


In [59]:
# Create Price Segment (Categorization)
logging.info("Creating price_segment feature")
def price_segment(price):
    if price < 300000:
        return "Budget"
    elif price < 700000:
        return "Mid"
    else:
        return "Premium"
df["price_segment"] = df["sold_price"].apply(price_segment)


2026-02-10 23:29:42,929 - INFO - Creating price_segment feature


In [60]:
# Transformation Validation
logging.info("Validating transformed data")

# Logical checks
df[df["vehicle_age"] < 0].shape[0], df[df["sold_price"] <= 0].shape[0]


2026-02-10 23:29:42,943 - INFO - Validating transformed data


(0, 0)

In [61]:
df[["manufacturing_year", "vehicle_age", "sold_price", "price_segment"]].head()


,manufacturing_year,vehicle_age,sold_price,price_segment
0,2015,11,889590,Premium
1,2019,7,1807733,Premium
2,2013,13,2240026,Premium
3,2015,11,2419873,Premium
4,2023,3,813106,Premium


I have completed:
✔ Extract
✔ Quality checks
✔ Cleaning
✔ Transformation & feature engineering
Data is now:
clean,
typed,
enriched,
analytics-ready


# SECTION 6 — CREATE DIMENSIONS & FACT

In [62]:
# Create dim_vehicle
# what was sold
#     -- removes repeated car details from the fact table.
logging.info("Creating dim_vehicle table")
dim_vehicle = (
    df[
        [
            "brand",
            "model",
            "fuel_type",
            "transmission",
            "car_condition",
            "is_premium_brand",
            "manufacturing_year",
            "vehicle_age",
            "kms_driven"
        ]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)

# Create surrogate key
dim_vehicle.insert(0, "vehicle_id", range(1, len(dim_vehicle) + 1))
dim_vehicle.head()



2026-02-10 23:29:42,976 - INFO - Creating dim_vehicle table


,vehicle_id,brand,model,fuel_type,transmission,car_condition,is_premium_brand,manufacturing_year,vehicle_age,kms_driven
0,1,Bmw,X3,Cng,Automatic,Poor,1,2015,11,162139
1,2,Hyundai,Verna,Electric,Manual,Fair,0,2019,7,57289
2,3,Honda,Amaze,Electric,Manual,Poor,0,2013,13,192873
3,4,Maruti,Baleno,Electric,Manual,Poor,0,2015,11,100035
4,5,Toyota,Innova,Petrol,Manual,Excellent,0,2023,3,120792


In [87]:
# Create dim_date and dim_location
import pandas as pd
import logging

logging.info("Creating dim_date table")

dim_date = (
    df[
        [
            "sold_date",
            "sold_year",
            "sold_month",
            "sold_month_num",
            "sold_month_name",
            "is_festive_season"
        ]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)

# surrogate key (optional but fine)
dim_date.insert(0, "date_id", range(1, len(dim_date) + 1))

# ✅ CRITICAL FIX — date_key
# ✅ SAFE date_key creation (handles bad dates)
dim_date["sold_date"] = pd.to_datetime(dim_date["sold_date"], errors="coerce")

# remove rows where sold_date is invalid
dim_date = dim_date.dropna(subset=["sold_date"]).reset_index(drop=True)

dim_date["date_key"] = (
    dim_date["sold_date"]
    .dt.strftime("%Y%m%d")
    .astype(int)
)


logging.info("Creating dim_location table")

dim_location = (
    df[
        [
            "seller_city",
            "seller_state"
        ]
    ]
    .drop_duplicates()
    .reset_index(drop=True)
)

dim_location.insert(0, "location_id", range(1, len(dim_location) + 1))

dim_date.head(), dim_location.head()



2026-02-11 00:15:11,962 - INFO - Creating dim_date table
2026-02-11 00:15:11,998 - INFO - Creating dim_location table


(   date_id  sold_date  sold_year  sold_month  sold_month_num sold_month_name  is_festive_season  date_key
 0        1 2021-11-03       2021          11               3             Mar                  1  20211103
 1        6 2022-09-12       2022           9              12             Dec                  0  20220912
 2        7 2022-02-01       2022           2               1             Jan                  0  20220201
 3        8 2023-11-08       2023           8               8             Aug                  0  20231108
 4        9 2021-12-05       2021          12               5             May                  0  20211205,
    location_id seller_city seller_state
 0            1        Pune    Rajasthan
 1            2      Mumbai       Punjab
 2            3   Bangalore  Maharashtra
 3            4  Chandigarh    Rajasthan
 4            5      Mumbai  Maharashtra)

In [89]:
# Create fact_vehicle_sales
logging.info("Creating fact_vehicle_sales table")

# Merge vehicle dimension
fact = df.merge(
    dim_vehicle,
    on=[
        "brand",
        "model",
        "fuel_type",
        "transmission",
        "car_condition",
        "is_premium_brand",
        "manufacturing_year",
        "vehicle_age",
        "kms_driven"
    ],
    how="left"
)

# Merge date dimension (ONLY for attributes, not key)
fact = fact.merge(
    dim_date,
    on=[
        "sold_date",
        "sold_year",
        "sold_month",
        "sold_month_num",
        "sold_month_name",
        "is_festive_season"
    ],
    how="left"
)

# Merge location dimension
fact = fact.merge(
    dim_location,
    on=[
        "seller_city",
        "seller_state"
    ],
    how="left"
)

# ✅ CRITICAL FIX — SAFE date_key creation
fact["sold_date"] = pd.to_datetime(fact["sold_date"], errors="coerce")
fact = fact.dropna(subset=["sold_date"]).reset_index(drop=True)

fact["date_key"] = (
    fact["sold_date"]
    .dt.strftime("%Y%m%d")
    .astype(int)
)

# ✅ FINAL FACT TABLE (NO date_id ANYWHERE)
fact_vehicle_sales = fact[
    [
        "vehicle_id",
        "date_key",          # ⭐ THIS IS THE JOIN KEY
        "location_id",
        "listed_price",
        "sold_price",
        "price_difference",
        "expected_market_price",
        "days_on_platform",
        "conversion_rate",
        "sale_flag"
    ]
]

fact_vehicle_sales.head()



2026-02-11 00:17:39,736 - INFO - Creating fact_vehicle_sales table


,vehicle_id,date_key,location_id,listed_price,sold_price,price_difference,expected_market_price,days_on_platform,conversion_rate,sale_flag
0,1,20211103,1,949951,889590,60361,474975,20,0.11,1
1,6,20220912,5,560789,523326,37463,364512,82,0.07,1
2,7,20220201,6,1086823,970701,116122,1032481,72,0.05,1
3,8,20231108,3,830991,671599,159392,747891,18,0.05,1
4,9,20211205,7,1538429,1407585,130844,692293,111,0.07,1


I have completed:
✔ Extract
✔ Quality checks
✔ Cleaning
✔ Transformation
✔ Dimensional modeling (FACT + DIMENSIONS)

In [98]:
fact_vehicle_sales.to_csv(
    "../processed/fact_vehicle_sales.csv",
    index=False
)

dim_date.to_csv(
    "../processed/dim_date.csv",
    index=False
)


# SECTION 7 — Validation & Metrics

In [65]:
# Key Integrity Checks
logging.info("Validating key integrity in fact table")

fact_vehicle_sales.isnull().sum()


2026-02-10 23:29:43,147 - INFO - Validating key integrity in fact table


vehicle_id                  0
date_id                  2964
location_id                 0
listed_price                0
sold_price                  0
price_difference            0
expected_market_price       0
days_on_platform            0
conversion_rate             0
sale_flag                   0
dtype: int64

In [66]:
# Row Count & Data Quality Metrics
logging.info("Validating row counts and data consistency")

rows_raw = df_raw.shape[0]
rows_clean = df.shape[0]
rows_fact = fact_vehicle_sales.shape[0]

rows_raw, rows_clean, rows_fact


2026-02-10 23:29:43,160 - INFO - Validating row counts and data consistency


(8000, 8000, 8000)

In [67]:
# Business Sanity Checks
logging.info("Running final business sanity checks")

# Check price distribution
fact_vehicle_sales[["listed_price", "sold_price"]].describe()


2026-02-10 23:29:43,173 - INFO - Running final business sanity checks


,listed_price,sold_price
count,8.000000e+03,8.000000e+03
mean,1.341170e+06,1.251477e+06
std,6.621348e+05,6.651735e+05
min,2.000180e+05,2.132400e+04
25%,7.693350e+05,6.764720e+05
50%,1.329258e+06,1.240210e+06
75%,1.912066e+06,1.820921e+06
max,2.498760e+06,2.493148e+06


In [68]:
# Ensure no negative values
fact_vehicle_sales[
    (fact_vehicle_sales["sold_price"] <= 0) |
    (fact_vehicle_sales["listed_price"] <= 0)
].shape[0]


0

I have completed:
✔ Extract
✔ Quality checks
✔ Cleaning
✔ Transformation
✔ Dimensional modeling
✔ Validation & metrics

# SECTION 8 — Save Outputs & Closure

In [69]:
# Save Dimension Tables (LOAD to Files)
logging.info("Saving dimension tables")

dim_vehicle.to_csv(
    PROCESSED_DATA_PATH + "dim_vehicle.csv",
    index=False
)

dim_date.to_csv(
    PROCESSED_DATA_PATH + "dim_date.csv",
    index=False
)

dim_location.to_csv(
    PROCESSED_DATA_PATH + "dim_location.csv",
    index=False
)

logging.info("Dimension tables saved successfully")


2026-02-10 23:29:43,213 - INFO - Saving dimension tables
2026-02-10 23:29:43,258 - INFO - Dimension tables saved successfully


In [70]:
# SQLITE CONNECTION
from sqlalchemy import create_engine

logging.info("Creating SQLite database connection")

engine = create_engine("sqlite:///used_car_analytics.db")

logging.info("SQLite database connected")



2026-02-10 23:29:43,477 - INFO - Creating SQLite database connection
2026-02-10 23:29:43,496 - INFO - SQLite database connected


In [71]:
# Load Tables into MySQL
logging.info("Loading tables into SQLite database")

dim_vehicle.to_sql("dim_vehicle", engine, if_exists="replace", index=False)
dim_date.to_sql("dim_date", engine, if_exists="replace", index=False)
dim_location.to_sql("dim_location", engine, if_exists="replace", index=False)
fact_vehicle_sales.to_sql("fact_vehicle_sales", engine, if_exists="replace", index=False)

logging.info("All tables loaded into SQLite successfully")


2026-02-10 23:29:43,506 - INFO - Loading tables into SQLite database
2026-02-10 23:29:43,751 - INFO - All tables loaded into SQLite successfully


In [72]:
import pandas as pd

pd.read_sql("SELECT COUNT(*) FROM fact_vehicle_sales", engine)


,COUNT(*)
0,8000


In [73]:
# Save Fact Table
logging.info("Saving fact table")

fact_vehicle_sales.to_csv(
    PROCESSED_DATA_PATH + "fact_vehicle_sales.csv",
    index=False
)

logging.info("Fact table saved successfully")


2026-02-10 23:29:43,776 - INFO - Saving fact table
2026-02-10 23:29:43,826 - INFO - Fact table saved successfully


In [74]:
# Pipeline Execution Summary
logging.info("ETL Pipeline Execution Summary")

print("Rows in raw data      :", df_raw.shape[0])
print("Rows after cleaning   :", df.shape[0])
print("Rows in fact table    :", fact_vehicle_sales.shape[0])

print("\nDimension sizes:")
print("Vehicles :", dim_vehicle.shape[0])
print("Dates    :", dim_date.shape[0])
print("Locations:", dim_location.shape[0])


2026-02-10 23:29:43,841 - INFO - ETL Pipeline Execution Summary


Rows in raw data      : 8000
Rows after cleaning   : 8000
Rows in fact table    : 8000

Dimension sizes:
Vehicles : 8000
Dates    : 2623
Locations: 30


## Conclusion

This project demonstrates a complete end-to-end ETL pipeline for a used car
sales dataset. Raw CSV data was extracted, validated, cleaned, transformed,
and modeled into analytics-ready fact and dimension tables using a star schema.

### Key Highlights
- Structured ETL workflow with logging and validation
- Business-driven feature engineering
- Proper dimensional modeling (vehicle, date, location)
- Analytics-ready outputs for SQL and BI tools

### Next Steps
- Load tables into a SQL database (MySQL / PostgreSQL)
- Build dashboards using Power BI or Tableau
- Add incremental ETL or automation in future


In [75]:
#SQLite DB Connection
from sqlalchemy import create_engine
import logging

logging.info("Creating SQLite database connection")

engine = create_engine("sqlite:///used_car_analytics.db")

logging.info("SQLite database connected")


2026-02-10 23:29:43,864 - INFO - Creating SQLite database connection
2026-02-10 23:29:43,866 - INFO - SQLite database connected


In [76]:
#Load Tables into SQLite (LOAD phase)
logging.info("Loading tables into SQLite database")

dim_vehicle.to_sql("dim_vehicle", engine, if_exists="replace", index=False)
dim_date.to_sql("dim_date", engine, if_exists="replace", index=False)
dim_location.to_sql("dim_location", engine, if_exists="replace", index=False)
fact_vehicle_sales.to_sql("fact_vehicle_sales", engine, if_exists="replace", index=False)

logging.info("All tables loaded into SQLite successfully")


2026-02-10 23:29:43,877 - INFO - Loading tables into SQLite database
2026-02-10 23:29:44,104 - INFO - All tables loaded into SQLite successfully


In [77]:
import pandas as pd

pd.read_sql(
    "SELECT COUNT(*) AS total_sales FROM fact_vehicle_sales",
    engine
)


,total_sales
0,8000


In [78]:
# Top 5 brands by total revenue
pd.read_sql("""
    SELECT v.brand,
           SUM(f.sold_price) AS total_revenue
    FROM fact_vehicle_sales f
    JOIN dim_vehicle v ON f.vehicle_id = v.vehicle_id
    GROUP BY v.brand
    ORDER BY total_revenue DESC
    LIMIT 5;
""", engine)


,brand,total_revenue
0,Hyundai,1315591543
1,Bmw,1303589662
2,Audi,1266405161
3,Tata,1258615694
4,Toyota,1245571056


In [91]:
fact_vehicle_sales.head()


,vehicle_id,date_key,location_id,listed_price,sold_price,price_difference,expected_market_price,days_on_platform,conversion_rate,sale_flag
0,1,20211103,1,949951,889590,60361,474975,20,0.11,1
1,6,20220912,5,560789,523326,37463,364512,82,0.07,1
2,7,20220201,6,1086823,970701,116122,1032481,72,0.05,1
3,8,20231108,3,830991,671599,159392,747891,18,0.05,1
4,9,20211205,7,1538429,1407585,130844,692293,111,0.07,1


In [92]:
print(df.shape)
print(dim_date.shape)
print(dim_location.shape)
print(fact_vehicle_sales.shape)


(8000, 32)
(2623, 8)
(30, 3)
(5036, 10)


## Sample Insights from SQL Analysis

- Certain brands contribute disproportionately to total revenue
- Premium segment vehicles show higher average selling price
- Sales volume varies significantly by city and month

These insights demonstrate how the ETL pipeline supports downstream
business analytics.
